In [ ]:
# Установка зависимостей
!pip install torch torchvision
!pip install numpy opencv-python

# Клонирование репозитория с реализацией SSD от Nvidia
!git clone https://github.com/NVIDIA/DeepLearningExamples.git
%cd DeepLearningExamples/PyTorch/Detection/SSD

# Подключение Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from src.model import SSD300, Loss
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Функция для сохранения модели
def save_model(epoch, model, path):
    filename = f'model_epoch_{epoch}.pth'
    torch.save(model.state_dict(), os.path.join(path, filename))
    print(f'Model saved: {filename}')

# Путь к вашему датасету на Google Drive
dataset_path = '/content/drive/My Drive/path_to_your_dataset'

# Настройка параметров, аналогичных YOLOv8
epochs = 70
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
img_size = 1080
iou_threshold = 0.8

# Создание и настройка модели
model = SSD300(pretrained_backbone=True).to(device)

# Оптимизатор Adam
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Функция потерь с учетом IoU
criterion = Loss(iou_threshold=iou_threshold)

# Функция для загрузки и преобразования данных
def get_data_loader(img_size, batch_size):
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        # Добавьте дополнительные преобразования при необходимости
    ])

    # Загрузка датасета
    dataset = datasets.ImageFolder(dataset_path, transform=transform)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return data_loader

# Обучение модели
def train(model, data_loader, optimizer, criterion, epochs, device, save_path):
    model.train()
    for epoch in range(epochs):
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)

            # Очистка градиентов
            optimizer.zero_grad()

            # Прямой проход
            outputs = model(images)

            # Расчет потерь
            loss = criterion(outputs, labels)

            # Обратный проход и оптимизация
            loss.backward()
            optimizer.step()

        # Сохранение модели после каждой эпохи
        save_model(epoch, model, save_path)

        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Подготовка данных и запуск обучения
data_loader = get_data_loader(img_size, batch_size=32)
model_save_path = '/content/drive/My Drive/Университет ИИ/Стажировка/Weights/YourModel'
train(model, data_loader, optimizer, criterion, epochs, device, model_save_path)
